<a href="https://colab.research.google.com/github/Atomix77/IMLO-IP/blob/main/IMLO_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import torch
# torch.cuda.is_available()

# Load Data



In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

trainingTransform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(20),
      transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

testTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

trainingData = datasets.Flowers102(
    root = "./datasets",
    split = "train",
    transform = trainingTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

validationData = datasets.Flowers102(
    root = "./datasets",
    split = "val",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

testData = datasets.Flowers102(
    root = "./datasets",
    split = "test",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

c:\Users\tomha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tomha\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tomha\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


# Dataloaders

In [3]:
batchSize = 64

trainingDataloader = DataLoader(trainingData, batchSize, shuffle = True)
testDataloader = DataLoader(testData, batchSize)

# Get device

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Create Aritecture

In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self, classAmount):
    super(NeuralNetwork, self).__init__()
    self.convStack =  nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.classifier = nn.Linear(128 * 28 * 28, classAmount)

  def forward(self, x):
    x = self.convStack(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

model = NeuralNetwork(102).to(device)

# Testing

In [6]:
def testModel(dataloader, model, lossFunction):
  print('Testing')
  model.eval()
  total = 0
  correct = 0
  testLoss = 0
  
  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      total += y.size(0)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  testLoss = testLoss/total
  correct = correct/total
  print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")

# Training

In [7]:
def trainingModel(dataloader, model, lossFunction, optimizer, epochs):
  model.train()
  for epoch in range(epochs):
    print(f'Epoch: {epoch+1}/{epochs}:')
    print('Training Model')
    currentLoss = 0.0
    correct = 0
    total = 0
    testLoss = 0
    
    for X, y in dataloader:
      optimizer.zero_grad()
      
      pred = model(X)
      loss = lossFunction(pred, y)

      loss.backward()
      optimizer.step()

      currentLoss += loss.item()

      total += y.size(0)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    testLoss = testLoss/total
    correct = correct/total
    print(f'Accuracy {(100*correct):>0.1f}%, Loss: {currentLoss / len(dataloader):.5f}')
    testModel(testDataloader, model, lossFunction)

In [8]:
learningRate = 0.001
batchSize = 128
epochs = 20

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

trainingModel(trainingDataloader, model, lossFunction, optimizer, epochs)
print("Finished")

Epoch: 1/20:
Training Model
Accuracy 1.0%, Loss: 5.74630
Testing
Accuracy: 3.0%, Avg loss: 0.069219 

Epoch: 2/20:
Training Model
Accuracy 9.8%, Loss: 3.97329
Testing
Accuracy: 11.2%, Avg loss: 0.060064 

Epoch: 3/20:
Training Model
Accuracy 22.7%, Loss: 3.19975
Testing
Accuracy: 15.7%, Avg loss: 0.060507 

Epoch: 4/20:
Training Model
Accuracy 36.0%, Loss: 2.60966
Testing
Accuracy: 19.9%, Avg loss: 0.060589 

Epoch: 5/20:
Training Model
Accuracy 46.7%, Loss: 2.13329
Testing
Accuracy: 18.0%, Avg loss: 0.065954 

Epoch: 6/20:
Training Model
Accuracy 49.6%, Loss: 1.94083
Testing
Accuracy: 19.4%, Avg loss: 0.064733 

Epoch: 7/20:
Training Model
Accuracy 57.0%, Loss: 1.68075
Testing
Accuracy: 19.3%, Avg loss: 0.071622 

Epoch: 8/20:
Training Model
Accuracy 61.4%, Loss: 1.47007
Testing
Accuracy: 20.2%, Avg loss: 0.076843 

Epoch: 9/20:
Training Model
Accuracy 67.2%, Loss: 1.24704
Testing
Accuracy: 21.1%, Avg loss: 0.080798 

Epoch: 10/20:
Training Model
Accuracy 67.8%, Loss: 1.17811
Testing


Current Best: Accuracy: 22.6%, Avg Loss: 0.110180

# Saving the model

In [9]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model, 'model.pth')

# Loading the model

In [10]:
model = models.vgg16()
model.load_state_dict(torch.load('model.pth'))
model.eval()

TypeError: Expected state_dict to be dict-like, got <class 'torchvision.models.vgg.VGG'>.